<a href="https://colab.research.google.com/github/JulioCastillo1/TP1-Datos/blob/master/Copia_de_Red_convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ARRANCAMOS CON LO BÁSICO
import numpy as np
import tensorflow as tf
import pandas as pd
import random
from google.colab import files

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [ ]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_tmp_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_tmp_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# LIMPIAMOS Y CHUSMEAMOS
train_tmp_df = train_tmp_df.drop(['keyword', 'location'], axis = 1)
train_tmp_df.head()
train_tmp_df.shape

(7613, 3)

In [ ]:
# EXTRAEMOS UN SET DE VALIDACIÓN DEL 10%
from sklearn.model_selection import train_test_split

train_df, validation_df = train_test_split(train_tmp_df, test_size = 0.1)
train_df.shape[0]

6090

In [ ]:
validation_df.shape[0]


1523

In [ ]:
# HACEMOS UN EMBEDDING PROVISTO POR UN "TensorFlow Hub model"

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_list = np.asarray(train_df['text'])
train_target_list = np.asarray(train_df['target'])

hub_layer(train_text_list[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[-1.3130846 , -0.1299771 , -0.6460433 , -1.2813138 , -0.35637245,
        -0.6728002 , -0.78708506, -1.2084562 , -0.69668853,  1.0260129 ,
        -1.053807  ,  0.1985695 ,  0.7754305 ,  0.06631742,  0.75959885,
        -0.1719684 , -1.8653511 ,  0.383707  ,  0.73699456,  0.091278  ],
       [ 0.17786609,  0.05419118,  0.20797475,  0.43611705,  0.24940072,
        -0.40636843, -1.0988255 ,  0.46651414,  0.38593405,  0.7438608 ,
        -1.9111044 ,  0.890076  , -0.441046  , -0.5248492 , -1.5569504 ,
         0.7120916 ,  0.33593795, -0.09674331, -1.0356532 ,  0.45747846],
       [-0.7684545 , -0.3240194 , -0.4590403 , -0.78238493, -0.6346923 ,
         0.84253484,  0.3933478 ,  0.3958523 , -0.12424278,  0.36349782,
        -0.243992  , -0.09113774,  0.36228633, -0.17937404,  0.5450773 ,
         0.85830885,  0.44275758, -0.47713774, -0.7886732 ,  0.7035698 ]],
      dtype=float32)>

In [ ]:
# ARMAMOS EL MODELO
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_9 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_20 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_21 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 17        
Total params: 400,645
Trainable params: 400,645
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# COMPILAMOS Y ENTRENAMOS

# NOTAS:
# Con 20 epochs obtiene mejor resultado en Kaggle que con 30 porque overfitearía

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_text_list, train_target_list, batch_size=512,
                    epochs = 20,
                    verbose = 1)

Epoch 1/20
12/12 [==============================] - 0s 11ms/step - loss: 0.7384 - accuracy: 0.5609
Epoch 2/20
12/12 [==============================] - 0s 10ms/step - loss: 0.6437 - accuracy: 0.5770
Epoch 3/20
12/12 [==============================] - 0s 10ms/step - loss: 0.5838 - accuracy: 0.6099
Epoch 4/20
12/12 [==============================] - 0s 11ms/step - loss: 0.5346 - accuracy: 0.6724
Epoch 5/20
12/12 [==============================] - 0s 10ms/step - loss: 0.4883 - accuracy: 0.7305
Epoch 6/20
12/12 [==============================] - 0s 10ms/step - loss: 0.4420 - accuracy: 0.7798
Epoch 7/20
12/12 [==============================] - 0s 11ms/step - loss: 0.3970 - accuracy: 0.8202
Epoch 8/20
12/12 [==============================] - 0s 11ms/step - loss: 0.3534 - accuracy: 0.8466
Epoch 9/20
12/12 [==============================] - 0s 10ms/step - loss: 0.3124 - accuracy: 0.8688
Epoch 10/20
12/12 [==============================] - 0s 11ms/step - loss: 0.2738 - accuracy: 0.8921
Epoch 11/

In [ ]:
# EVALUAMOS CON EL SET DE VALIDACION
validation_text_list = np.asarray(train_df['text'])
validation_target_list = np.asarray(train_df['target'])

loss, precision = model.evaluate(validation_text_list, validation_target_list)
print('Precisión: %.2f' % (precision*100))

191/191 [==============================] - 0s 2ms/step - loss: 0.0987 - accuracy: 0.9626
Precisión: 96.26


In [ ]:
# ARMAMOS LA PREDICCIÓN
test_text_list = np.asarray(test_df['text'])
prediction = model.predict_classes(test_text_list)
prediction

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
# EXPORTAMOS EL CSV EN EL FORMATO QUE PIDE KAGGLE
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>